In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
import imageio

In [ ]:
path = Path('../input/Kannada-MNIST')
train = pd.read_csv('../input/Kannada-MNIST/train.csv')
add_b4_mnist = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')
test  = pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
print('train' + str(train.shape) + '\n' + 'test' +  str(test.shape) + '\n' + 'val' + str(add_b4_mnist.shape))

In [ ]:
def to_img_shape(data_X, data_y=[]):
    data_X = np.array(data_X).reshape(-1,28,28)
    #data_X = data_X.astype('float32')
    #data_X = data_X / 255.0
    data_X = np.stack((data_X,)*3, axis=-1)
    data_y = np.array(data_y)
    return data_X,data_y

In [ ]:
train.head()

In [ ]:
add_b4_mnist.head()

In [ ]:
test.head()

In [ ]:
train_X, train_y = train.loc[:,'pixel0':'pixel783'], train['label']
sub_val_X, sub_val_y = add_b4_mnist.loc[:,'pixel0':'pixel783'], add_b4_mnist['label']
test_X,test_idx = test.loc[:,'pixel0':'pixel783'], test['id']

In [ ]:
train_X,train_y = to_img_shape(train_X, train_y)
sub_val_X,sub_val_y = to_img_shape(sub_val_X,sub_val_y)
test_X,test_idx = to_img_shape(test_X,test_idx)

In [ ]:
print('train' + str(train_X.shape) + '\n' + 'val' + str(sub_val_X.shape)+'\n'+'test'+str(test_X.shape))

In [ ]:
plt.title(f'Label: {train_y[55]}')
plt.imshow(train_X[55],cmap='gray');

In [ ]:
def save_imgs(path:Path, data, labels):
    path.mkdir(parents=True,exist_ok=True)
    for label in np.unique(labels):
        (path/str(label)).mkdir(parents=True,exist_ok=True)
    for i in range(len(data)):
        if(len(labels)!=0):
            imageio.imsave( str( path/str(labels[i])/(str(i)+'.jpg') ), data[i].astype(np.uint8) )
        else:
            imageio.imsave( str( path/(str(i)+'.jpg') ), data[i].astype(np.uint8) )

In [ ]:
save_imgs(Path('/data/train'),train_X,train_y)
save_imgs(Path(DATAPATH/'add_train'),sub_val_X,sub_val_y)
save_imgs(Path(DATAPATH/'test_sub'),test_X,test_idx)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [ ]:
tfms = get_transforms(flip_vert=False)

In [ ]:
data = (ImageList.from_folder('/data/train') 
        .split_by_rand_pct(0.2)       
        .label_from_folder()        
        .add_test_folder()          
        .transform(tfms, size=64)   
        .databunch().normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(6,6))

In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints
!cp /kaggle/input/resnet5019c8e357/resnet50-19c8e357.pth /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate,accuracy], model_dir = Path('../kaggle/working'),path = Path("."))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 0.01

In [ ]:
learn.fit_one_cycle(10,slice(lr,2*lr))